<a href="https://colab.research.google.com/github/Sudhandar/Adding-relevant-emoticons/blob/master/bert_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
!pip install pytorch-pretrained-bert pytorch-nlp

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [16]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv
Saving test.csv to test (1).csv


In [0]:
train = pd.read_csv('train (1).csv')
test = pd.read_csv('test (1).csv')

In [20]:
train.head(5)

,sentence,label
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [0]:
sentences = train.sentence.values
sentences = [ "[CLS] "+sentence+" [SEP]" for sentence in sentences]

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

tokenized_text = [tokenizer.tokenize(sentence) for sentence in sentences]

tokenized_text[0]

['[CLS]', 'never', 'talk', 'to', 'me', 'again', '[SEP]']

In [0]:
max_len = 50
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
input_ids = pad_sequences(input_ids,maxlen=max_len,dtype='long',padding='post',truncating='post')

In [0]:
attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [33]:
train_labels = pd.get_dummies(train['label'])
labels = train_labels.values
labels[0]

array([0, 0, 0, 1, 0], dtype=uint8)

In [0]:
def bert_inputs(df,maxlen):
  sentences = df.sentence.values
  sentences = ["[CLS] "+sentence+" [SEP]" for sentence in sentences]
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)
  sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
  sentence_ids = [tokenizer.convert_tokens_to_ids(sentence) for sentence in sentences]
  sentence_ids = pad_sequences(sentence_ids,maxlen=maxlen,dtype='long',padding='post',truncating='post')
  attention_masks = []
  for sentence in sentence_ids:
    seq_mask = [float(i>0) for i in sentence]
    attention_masks.append(seq_mask)
  lables = pd.get_dummies(df['label'])
  labels = lables.values
  return sentence_ids,labels,attention_masks


In [0]:
test_inputs, test_labels, test_masks = bert_inputs(test,50)

In [0]:
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
test_inputs = torch.tensor(test_inputs)
test_labels = torch.tensor(test_labels)

tensor([ 101, 2196, 2831, 2000, 2033, 2153,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])